# Acquiring and Processing Information on the World's Largest Banks

## Project Scenario:
You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

Particulars of the code to be made have been shared below.

| Parameter                         | Value                                                                                                        |
|-----------------------------------|--------------------------------------------------------------------------------------------------------------|
| Code name                         | banks_project.py                                                                                             |
| Data URL                          | [https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks](https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks)  |
| Exchange rate CSV path            | [Exchange Rate CSV](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv)             |
| Table Attributes (upon Extraction only) | Name, MC_USD_Billion                                                                                        |
| Table Attributes (final)          | Name, MC_USD_Billion, MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion                                          |
| Output CSV Path                   | ./Largest_banks_data.csv                                                                                     |
| Database name                     | Banks.db                                                                                                     |
| Table name                        | Largest_banks                                                                                                |
| Log file                          | code_log.txt                                                                                                 |


## Project tasks

**Task 1:**
Write a function **``log_progress()``** to log the progress of the code at different stages in a file **``code_log.txt``**. Use the list of log points provided to create log entries as every stage of the code.

**Task 2:**
Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
 - a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
 - b. Write the code for a function **``extract()``** to perform the required data extraction.
 - c. Execute a function call to **``extract()``** to verify the output.

**Task 3:**
Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
a. Write the code for a function **``transform()``** to perform the said task.
b. Execute a function call to **``transform()``** and verify the output.

**Task 4:**
Load the transformed dataframe to an output CSV file. Write a function **``load_to_csv()``**, execute a function call and verify the output.

**Task 5:**
Load the transformed dataframe to an SQL database server as a table. Write a function **``load_to_db()``**, execute a function call and verify the output.

**Task 6:**
Run queries on the database table. Write a function **``load_to_db()``**, execute a given set of queries and verify the output.

**Task 7:**
Verify that the log entries have been completed at all stages by checking the contents of the file **``code_log.txt``**.




# Preliminaries: Installing Libraries and Downloading Data

Before building the code, you need to install the required libraries.

## The libraries needed for the code are:

- **requests** - The library used for accessing the information from the URL.
- **bs4** - The library containing the `BeautifulSoup` function used for web scraping.
- **pandas** - The library used for processing the extracted data, storing it in required formats, and communicating with the databases.
- **sqlite3** - The library required to create a database server connection.
- **numpy** - The library required for the mathematical rounding operations.
- **datetime** - The library containing the `datetime` function used for extracting the timestamp for logging purposes.


### Importing the required libraries

In [111]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
import sqlite3
from bs4 import BeautifulSoup

### Initialize all the known entitie

Here, you define the required entities and call the relevant
functions in the correct order to complete the project.

In [112]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attr = ["Name","MC_USD_Billion"]
output_csv_path = "./Largest_banks_data.csv"
db_name = "Banks.db"
table_name = "Largest_banks"
log_file_path = "./code_log.txt"

query_smt = """ SELECT * FROM Largest_banks """
query_smt2 = """ SELECT AVG(MC_GBP_Billions) FROM Largest_banks """
query_smt3 = """ SELECT Name from Largest_banks LIMIT 5 """

exchange_rate = pd.read_csv('exchange_rate.csv')

### Task 1: Logging function

Each log entry must happen in the next line in the text file.

You must associate the correct log entries with each of the executed function calls. Use the following table to note the logging message at the end of each function call that follows.

| **Task**                          | **Log message on completion**                                                 |
|-----------------------------------|-------------------------------------------------------------------------------|
| Declaring known values             | Preliminaries complete. Initiating ETL process                                 |
| Call `extract()` function          | Data extraction complete. Initiating Transformation process                    |
| Call `transform()` function        | Data transformation complete. Initiating Loading process                       |
| Call `load_to_csv()`               | Data saved to CSV file                                                         |
| Initiate SQLite3 connection        | SQL Connection initiated                                                      |
| Call `load_to_db()`                | Data loaded to Database as a table, Executing queries                          |
| Call `run_query()`                 | Process Complete                                                              |
| Close SQLite3 connection           | Server Connection closed                                                      |

At this stage, you should now make the first log entry from the table above.


In [113]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    timestamp_format = '%Y-%m-%d-%H:%M:%S'
    now = dt.datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open("./etl_project_log.txt", 'a') as f:
        f.write(timestamp + ' : ' + message + '\n')


### Task 2 : Extraction of data

In [114]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    html_page = requests.get(url).text
    data = BeautifulSoup(html_page, "html.parser")
    df = pd.DataFrame(columns=table_attr)

    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')

    for row in rows:
        col = row.find_all('td')

        if len(col) >0:
            data_dict = {"Name":col[1].get_text(strip = True),
                        "MC_USD_Billion": col[2].get_text(strip = True)}
            df1 = pd.DataFrame(data_dict, index = [0])
            df = pd.concat([df, df1], ignore_index= True)
     
    return df

#extract(url, table_attr).head()
#extracted_df = extract(url, table_attr)

### Task 3 : Transformation of data

In [115]:
def transform(df, exchange_rate):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies.

    Columns to add:
        MC_GBP_Billion, MC_EUR_Billion and MC_INR_Billion

    Table Attributes (final):
        Name, MC_USD_Billion, MC_GBP_Billion, MC_EUR_Billion, MC_INR_Billion
    '''
    df = df.astype({"Name":"string","MC_USD_Billion":"float"})

    for _, row in exchange_rate.iterrows():
      df[f'MC_{row["Currency"]}_Billions'] = np.round(df["MC_USD_Billion"] * row['Rate'], 2) 

    return df

#transformed_df = transform(extracted_df, exchange_rate)
#transformed_df['MC_EUR_Billions'][4]

In [84]:
#transform(extracted_df, exchange_rate).head()

,Name,MC_USD_Billion,MC_EUR_Billions,MC_GBP_Billions,MC_INR_Billions
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63


### Task 4: Loading to CSV

In [116]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    if len(df) > 0:
        df.to_csv(output_path)
    else:
        print("Dataframe is empty!. The file wasn't created")

#load_to_csv(transformed_df, output_csv_path)

### Task 5: Loading to Database

In [117]:
def load_to_db(df, sql_connection, table_name):
    '''This function saves the final dataframe as a database table
       with the provided name. Function returns nothing.'''
    #sql_connection = db_name
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)


### Task 6: Function to Run queries on Database

In [118]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

### Task 7: Verify log entries

In [122]:
# Error handling
sql_connection = sqlite3.connect('World_Economies.db')
try:
    log_progress('Preliminaries complete. Initiating ETL process')

    # Data Extraction
    try:
        df_extracted = extract(url, table_attr)
        log_progress('Data extraction complete. Initiating Transformation process')
    except Exception as e:
        log_progress(f'Error during data extraction: {str(e)}')
        raise

    # Data Transformation
    try:
        df_transformed = transform(df_extracted, exchange_rate)
        log_progress('Data transformation complete. Initiating loading process')
    except Exception as e:
        log_progress(f'Error during data transformation: {str(e)}')
        raise

    # Loading data to csv
    try:
        load_to_csv(df_transformed, csv_output_path)
        log_progress('Data saved to CSV file')
    except Exception as e:
        log_progress(f'Error saving data to CSV: {str(e)}')
        raise

    # Connecting to Database
    try:
        sql_connection = sqlite3.connect('World_Economies.db')
        log_progress('SQL Connection initiated.')
    except Exception as e:
        log_progress(f'Error initiating SQL connection: {str(e)}')
        raise

    # Loading data to the Database
    try:
        load_to_db(df_transformed, sql_connection, table_name)
        log_progress('Data loaded to Database as table. Running the query')
    except Exception as e:
        log_progress(f'Error loading data to database: {str(e)}')
        raise

    # Execute queries
    try:
        run_query(query_smt, sql_connection)
        log_progress('Process Complete.')
    except Exception as e:
        log_progress(f'Error running SQL query: {str(e)}')
        raise

    try:
        run_query(query_smt2, sql_connection)
        log_progress('Process Complete.')
    except Exception as e:
        log_progress(f'Error running SQL query: {str(e)}')
        raise

    try:
        run_query(query_smt3, sql_connection)
        log_progress('Process Complete.')
    except Exception as e:
        log_progress(f'Error running SQL query: {str(e)}')
        raise

finally:
    # Cierre seguro de la conexión
    try:
        if sql_connection:
            sql_connection.close()
            log_progress('SQL connection closed.')
    except Exception as e:
        log_progress(f'Error closing SQL connection: {str(e)}')

 SELECT * FROM Largest_banks 
                                      Name  MC_USD_Billion  MC_EUR_Billions  \
0                           JPMorgan Chase          432.92           402.62   
1                          Bank of America          231.52           215.31   
2  Industrial and Commercial Bank of China          194.56           180.94   
3               Agricultural Bank of China          160.68           149.43   
4                                HDFC Bank          157.91           146.86   
5                              Wells Fargo          155.87           144.96   
6                        HSBC Holdings PLC          148.90           138.48   
7                           Morgan Stanley          140.83           130.97   
8                  China Construction Bank          139.82           130.03   
9                            Bank of China          136.81           127.23   

   MC_GBP_Billions  MC_INR_Billions  
0           346.34         35910.71  
1           185.22      

In [123]:
df_extracted.head()

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91


In [124]:
df_transformed.head()

,Name,MC_USD_Billion,MC_EUR_Billions,MC_GBP_Billions,MC_INR_Billions
0,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,Bank of America,231.52,215.31,185.22,19204.58
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,HDFC Bank,157.91,146.86,126.33,13098.63
